In [1]:
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

from gymnasium.envs.registration import register

from gymnasium import spaces
import torch as th
from torch import nn
import numpy as np
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

import gymnasium as gym
import os

from gym_data.envs import CarcassoneEnv
# device = torch.device("cuda")
register(id='Carcassone-v0',entry_point='gym_data.envs:CarcassoneEnv',) 

2023-09-05 13:59:29.269720: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-05 13:59:29.763280: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment ALE/Adventure-v5 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment ALE/Adventure-ram-v5 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environm

In [2]:
extractable = ["image", "big_farmer_planes", "big_meeples_planes","chapel_plane", "city_planes", "farmer_planes", "field_planes", 
               "flowers_plane", "meeple_planes", "other_properties_plane", "road_planes", "shield_plane", "abbot_planes"]
class CustomCombinedExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: spaces.Dict):
        # We do not know features-dim here before going over all the items,
        # so put something dummy for now. PyTorch requires calling
        # nn.Module.__init__ before adding modules
        # print(observation_space)
        super().__init__(observation_space, features_dim=1)

        extractors = {}

        total_concat_size = 128
        # We need to know size of the output of this extractor,
        # so go over all the spaces and compute output feature sizes
        for key, subspace in observation_space.spaces.items():
            if key == "image":
                # We will just downsample one channel of the image by 4x4 and flatten.
                # Assume the image is single-channel (subspace.shape[0] == 0)
                extractors[key] = nn.Sequential(nn.MaxPool2d(2), nn.Flatten())
                total_concat_size += subspace.shape[1] // 2 * subspace.shape[2] // 2
            elif key == "vector":
                # Run through a simple MLP
                extractors[key] = nn.Linear(subspace.shape[0], 128)
                total_concat_size += 128
            elif key == "big_farmer_planes":
                shape =  subspace.shape[1] * subspace.shape[2] * subspace.shape[3]
                extractors[key] = nn.Sequential(nn.Flatten(), nn.Linear(shape, 512), nn.Linear(512, 128)) 
                total_concat_size += 128
            elif key == "big_meeples_planes":
                shape =  subspace.shape[1] * subspace.shape[2] * subspace.shape[3]
                extractors[key] = nn.Sequential(nn.Flatten(), nn.Linear(shape, 512), nn.Linear(512, 128)) 
                total_concat_size += 128
            elif key == "chapel_plane":
                shape =  subspace.shape[1] * subspace.shape[2] * subspace.shape[3]
                extractors[key] = nn.Sequential(nn.Flatten(), nn.Linear(shape, 128)) 
                total_concat_size += 128
            elif key == "city_planes":
                shape =  subspace.shape[1] * subspace.shape[2] * subspace.shape[3]
                extractors[key] = nn.Sequential(nn.Flatten(), nn.Linear(shape, 512), nn.Linear(512, 128)) 
                total_concat_size += 128
            elif key == "farmer_planes":
                shape =  subspace.shape[1] * subspace.shape[2] * subspace.shape[3]
                extractors[key] = nn.Sequential(nn.Flatten(), nn.Linear(shape, 512), nn.Linear(512, 128)) 
                total_concat_size += 128
            elif key == "field_planes":
                shape =  subspace.shape[1] * subspace.shape[2] * subspace.shape[3]
                extractors[key] = nn.Sequential(nn.Flatten(), nn.Linear(shape, 4096), nn.Linear(4096, 2048), nn.Linear(2048, 512), nn.Linear(512, 128)) 
                total_concat_size += 128
            elif key == "flowers_plane":
                shape =  subspace.shape[1] * subspace.shape[2] * subspace.shape[3]
                extractors[key] = nn.Sequential(nn.Flatten(), nn.Linear(shape, 128)) 
                total_concat_size += 128
            elif key == "meeple_planes":
                shape =  subspace.shape[1] * subspace.shape[2] * subspace.shape[3]
                extractors[key] = nn.Sequential(nn.Flatten(), nn.Linear(shape, 512), nn.Linear(512, 128)) 
                total_concat_size += 128
            elif key == "other_properties_plane":
                extractors[key] = nn.Linear(subspace.shape[0], 128)
                total_concat_size += 128
            elif key == "road_planes":
                shape =  subspace.shape[1] * subspace.shape[2] * subspace.shape[3]
                extractors[key] = nn.Sequential(nn.Flatten(), nn.Linear(shape, 512), nn.Linear(512, 128)) 
                total_concat_size += 128
            elif key == "shield_plane":
                shape =  subspace.shape[1] * subspace.shape[2] * subspace.shape[3]
                extractors[key] = nn.Sequential(nn.Flatten(), nn.Linear(shape, 128)) 
                total_concat_size += 128
            elif key == "abbot_planes":
                shape =  subspace.shape[1] * subspace.shape[2] * subspace.shape[3]
                extractors[key] = nn.Sequential(nn.Flatten(), nn.Linear(shape, 128)) 
                total_concat_size += 128

        self.extractors = nn.ModuleDict(extractors)
        print("Total concat size: " , total_concat_size)
        # Update the features dim manually
        self._features_dim = total_concat_size
#         print(total_concat_size)
    def forward(self, observations) -> th.Tensor:
        encoded_tensor_list = []

        # self.extractors contain nn.Modules that do all the processing.
        for key, extractor in self.extractors.items():
            # print(key)
            # print(observations[key].shape)
            extractor_value = extractor(observations[key])
            # if key == "other_properties_plane":
                # print("Extractor value: ",  extractor_value.shape)
            encoded_tensor_list.append(extractor_value)
        # Return a (B, self._features_dim) PyTorch tensor, where B is batch dimension.
        concated = th.cat(encoded_tensor_list, dim=1)
        # print("Forward total concat size: " , concated.shape)

        return concated

In [3]:
policy_kwargs = dict(
    features_extractor_class=CustomCombinedExtractor,
    net_arch = [1024, dict(pi=[256, 128], vf=[512, 64])]
)

def mask_fn(env: gym.Env) -> np.ndarray:
    # Do whatever you'd like in this function to return the action mask
    # for the current env. In this example, we assume the env has a
    # helpful method we can rely on.
    return env.valid_action_mask()


def make_env():
    env = CarcassoneEnv()
    env = ActionMasker(env, mask_fn)
    return env


# env = ActionMasker(env, mask_fn) 
env = make_vec_env(make_env,seed=1, n_envs=32)


In [4]:
import tensorboard


In [5]:
# from tensorboard import notebook
# notebook.list()

In [6]:
%load_ext tensorboard

In [7]:
%tensorboard --logdir ./  --bind_all --port 8080 

In [8]:
def linear_schedule(initial_value):
    """
    Linear learning rate schedule.
    :param initial_value: (float or str)
    :return: (function)
    """
    if isinstance(initial_value, str):
        initial_value = float(initial_value)

    def func(progress):
        """
        Progress will decrease from 1 (beginning) to 0
        :param progress: (float)
        :return: (float)
        """
        return progress * initial_value

    return func

In [ ]:
iter = 0

while True:
    print("iter", iter)
    env = make_vec_env(CarcassoneEnv, seed=1,  n_envs=16)

    # model = A2C("MultiInputPolicy", env, verbose=1,tensorboard_log="runs/", device="auto", n_steps=5, learning_rate=linear_schedule(0.0001))
    model = A2C.load("model_carcassone", env, verbose=1,tensorboard_log="runs/", device="auto", n_steps=5, learning_rate=linear_schedule(0.0001))
    import time
    # model.set_parameters("model_carcassone")
    start = time.time()
    try:
        model.learn(total_timesteps=2e8)
    except KeyboardInterrupt:
        interrupted = True
    except RuntimeError:
        end = time.time()
        
        elapsed_time = end - start
        
        print("Elapsed time: ", elapsed_time // 60, " minutes")
        
        model.save("model_carcassone")
        
        continue

    end = time.time()
    
    elapsed_time = end - start
    
    print("Elapsed time: ", elapsed_time // 60, " minutes")
    
    model.save("model_carcassone")
    iter +=1

iter 0
Logging to runs/A2C_108
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Valid
Valid
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Game finished
Valid
Valid
-------------------------------------
| rollout/ 

In [ ]:
model.save("model_carcassone")